In [1]:
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
import seaborn as sns

In [368]:
df=pd.read_csv('mon_data.csv')
df.dropna(subset=['text_image'],inplace=True)

In [369]:
df=df.iloc[:,[-1,3]]

In [370]:
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
stop_words.update(["." , ",",':'])
stop_words=list(stop_words)
stop_words.extend(stopwords.words('french'))

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(str(w).lower().strip())
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!]+", " ", w)
    w = re.sub(r'\b\w{0,2}\b', '', w)

    # remove stopword
    mots = word_tokenize(w.strip())
    mots = [mot for mot in mots if mot not in stop_words]
    return ' '.join(mots).strip()

In [371]:
df.text_image = df.text_image.apply(lambda x :preprocess_sentence(x))

In [409]:
from sklearn.model_selection import train_test_split
X_text_train,X_text_test, y_train, y_test=train_test_split(df.text_image,df.type,test_size=0.2,random_state=2345)

In [348]:
X_text_train

1599    resnik elchorn technology transfer april attac...
740                                                takere
189      permis conduire caxe ecimen testa bbed fra jules
890     criteria allowance travel expenses chairmen vi...
1471    soztal und pravenlivmeditin sociale objektive ...
                              ...                        
1299    biographical sketch daniel podolsky date birth...
1465                                                  nee
902     copy rough leber katz partners advertising dat...
1328    cigarette specification louisville shield ligh...
1142    thf bann sgon ncounoftce check date sac amount...
Name: text_image, Length: 1252, dtype: object

In [410]:
dic_count={}
for i in range(y_train.max()+1):
    dic_count[i]=[]

for i,txt in enumerate(X_text_train):
    dic_count[list(y_train)[i]].extend(txt.split())
dic_count_total={}
for key in  dic_count:
    dic_count_total[key]=pd.DataFrame(pd.DataFrame(dic_count[key]).value_counts()[:16],columns=['nbr'])
    

In [350]:
dic_count_total[6]#.value_counts()

,nbr
0,
source,172
date,120
www,110
new,106
https,103
industrydocuments,103
year,102
total,80
one,74


In [411]:
liste_len=[]
for i in dic_count_total:
    liste_len.append(dic_count_total[i].value_counts().shape[0])
liste_len    

[13,
 14,
 11,
 13,
 15,
 12,
 13,
 5,
 12,
 7,
 8,
 13,
 10,
 4,
 11,
 8,
 10,
 11,
 8,
 1,
 7,
 9,
 12]

In [412]:
list_index=[]
for key in dic_count_total:
    list_index.append([i[0] for i in dic_count_total[key].index])


In [376]:
 list_index

[['facture',
  'total',
  'tva',
  'montant',
  'date',
  'nom',
  'adresse',
  'paiement',
  'description',
  'rue',
  'numero',
  'prix',
  'client',
  'ttc',
  'paris'],
 ['nom',
  'francaise',
  'carte',
  'republique',
  'date',
  'nationale',
  'paris',
  'martin',
  'sexe',
  'identite',
  'fra',
  'nationalite',
  'usage',
  'signature',
  'marie'],
 ['deplacements',
  'attestation',
  'edf',
  'service',
  'cas',
  'deplacement',
  'domicile',
  'residence',
  'contrat',
  'lieu',
  'facture',
  'nom',
  'etre',
  'electricite',
  'plus'],
 ['passport',
  'passeport',
  'moviles',
  'date',
  'canada',
  'name',
  'bearer',
  'nom',
  'united',
  'jan',
  'union',
  'birth',
  'com',
  'pages',
  'pass'],
 ['valeur',
  'net',
  'revenu',
  'cotisations',
  'salaire',
  'total',
  'sociale',
  'base',
  'taux',
  'paie',
  'securite',
  'impot',
  'csg',
  'employeur',
  'bulletin'],
 ['carte',
  'postale',
  '!',
  'merci',
  'france',
  '?',
  'plus',
  'adresse',
  'cette',


In [401]:
dic_count_total[0].loc['facture']['nbr'].values[0]

151

In [413]:
for key in dic_count_total:
    dic_count_total[key]['1']=dic_count_total[key]/dic_count_total[key].sum()   

In [403]:
dic_count_total[0]

,nbr,1
0,,
facture,151,0.121676
total,140,0.112812
tva,106,0.085415
montant,94,0.075745
date,71,0.057212
nom,71,0.057212
adresse,63,0.050766
paiement,59,0.047542
description,51,0.041096


In [414]:
pred_sum=0
dic_pred_total={}
dic_pred={}
for n_exemple,txt in  enumerate(X_text_test):
    for mots in txt.split():
        for key , liste in enumerate(list_index):
            if mots in liste:
                    pred_sum=pred_sum+dic_count_total[key].loc[mots]['1'].values[0]
                    dic_pred[key]=pred_sum
                    
            else:
                continue
    dic_pred_total[n_exemple]=dic_pred
    dic_pred={}
    pred_sum=0
            
        
    

In [415]:
y_pred=[]
for key in dic_pred_total:
    if dic_pred_total[key]=={}:
        y_pred.append(27)
    else:
        key_list = [k  for (k, val) in dic_pred_total[key].items() if val == max(dic_pred_total[key].values())]
        y_pred.append(key_list[0])
        
    

In [416]:
y_pred=np.array(y_pred)
y_test=np.array(y_test)
index_list=np.where(y_pred==27)
y_test = np.delete(y_test, index_list)
y_pred = np.delete(y_pred, index_list)

In [417]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))
pd.crosstab(y_test,y_pred)

0.5333333333333333


col_0,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,20,21,22
row_0,,,,,,,,,,,,,,,,,,,,,
0,6,0,0,1,1,1,0,0,0,0,...,2,0,0,0,1,0,0,0,0,0
1,0,14,0,4,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,3,0
2,0,0,3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,2,0,9,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
4,3,1,2,0,26,0,1,0,0,1,...,0,0,0,0,3,0,0,0,0,0
5,0,2,0,0,0,7,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
6,0,1,0,2,0,0,53,1,1,1,...,0,2,0,1,1,0,0,0,4,0
7,0,0,0,0,0,0,0,1,0,0,...,0,1,0,1,0,0,0,1,1,1
8,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
